<a href="https://colab.research.google.com/github/plaban1981/Deep-Learning-KERAS/blob/master/UFC_Anomaly_dataset_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import os
import numpy as np
import glob
import cv2
import pandas as pd
import math

In [6]:
anomaly_df = pd.read_csv("/content/drive/MyDrive/UTC_Anamoly_dataset/Anomaly_Train.txt",delimiter="\t",header=None)

In [7]:
anomaly_df.head()

,0
0,Abuse/Abuse001_x264.mp4
1,Abuse/Abuse002_x264.mp4
2,Abuse/Abuse003_x264.mp4
3,Abuse/Abuse004_x264.mp4
4,Abuse/Abuse005_x264.mp4


1. Anomaly Detection Experiment

- A) Videos:
* Anomaly-Videos-Part-1-- Part-4 (4 zip files)
* Training-Normal-Videos-Part-1.zip and 
* Training-Normal-Videos-Part-2.zip (normal videos for training)
* Testing_Normal_Videos.zip (normal videos for testing)

* UCF_Crimes-Train-Test-Split.zip contains the traing and testing split in our experiments
(folder: Anomaly_Detection_splits)


- B) Temporal Annotations for Testing Videos (anomaly videos)

#### Temporal_Anomaly_Annotation_for_Testing_Videos.txt

 Each row of 'Temporal_Anomaly_Annotation.txt' is the annotation for a video, for example:
Abuse028_x264.mp4  Abuse  165  240  -1 -1  
-	The first column is the name of the video
-	The second column is the name of the anomalous event
-	The third column is the starting frame of the event (you will have to convert each video to image frames first) 
-	The fourth column is the ending frame of the event.
-	For videos in which second  instance of event occurs, fifth and sixth contains starting and ending frames of second instance.  
- Negative number means no anomalous event instance. In this example, abuse (instance) only occurs once.

Note: Ours videos have 30 frames per second.

In [24]:
annot_df = pd.read_csv("/content/drive/MyDrive/UTC_Anamoly_dataset/Temporal_Anomaly_Annotation_for_Testing_Videos.txt",
                       delim_whitespace=True,
                       names=['FileName','Event','Start_Frame1','End_Frame1','Start_Frame2','End_Frame2'])

In [25]:
annot_df.shape

(290, 6)

In [26]:
annot_df.head()

,FileName,Event,Start_Frame1,End_Frame1,Start_Frame2,End_Frame2
0,Abuse028_x264.mp4,Abuse,165,240,-1,-1
1,Abuse030_x264.mp4,Abuse,1275,1360,-1,-1
2,Arrest001_x264.mp4,Arrest,1185,1485,-1,-1
3,Arrest007_x264.mp4,Arrest,1530,2160,-1,-1
4,Arrest024_x264.mp4,Arrest,1005,3105,-1,-1


In [27]:
annot_df.tail()

,FileName,Event,Start_Frame1,End_Frame1,Start_Frame2,End_Frame2
285,Vandalism007_x264.mp4,Vandalism,240,750,-1,-1
286,Vandalism015_x264.mp4,Vandalism,2010,2700,-1,-1
287,Vandalism017_x264.mp4,Vandalism,270,330,780,840
288,Vandalism028_x264.mp4,Vandalism,1830,1980,2400,2670
289,Vandalism036_x264.mp4,Vandalism,540,780,990,1080


In [28]:
annot_df.to_csv("/content/drive/MyDrive/UTC_Anamoly_dataset/Annotation.csv",index=False)

In [37]:
annot_df[annot_df['FileName'] == "Normal_Videos_929_x264.mp4"]

,FileName,Event,Start_Frame1,End_Frame1,Start_Frame2,End_Frame2
193,Normal_Videos_929_x264.mp4,Normal,-1,-1,-1,-1


## Unzip videos

In [36]:
! unzip "/content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition (1).zip" -d "/content/drive/MyDrive/UTC_Anamoly_dataset/Normal"

Archive:  /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition (1).zip
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_015_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_050_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_100_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_129_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_150_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_246_x264.mp4  
  inflating: /content/drive/MyDrive/UTC_Anamoly_dataset/Normal/Normal_Videos_for_Event_Recognition/Normal_Videos_247_x264.mp4  


In [40]:
for folder in os.listdir("/content/drive/MyDrive/UTC_Anamoly_dataset/Anomaly"):
  print(folder)
  for sub in os.listdir(os.path.join("/content/drive/MyDrive/UTC_Anamoly_dataset/Anomaly",folder)):
    print(sub)

Anomaly-Videos-Part-1
Abuse
Assault
Arrest
Arson
Anomaly-Videos-Part-2
Burglary
Fighting
Explosion
Anomaly-Videos-Part-3
Robbery
RoadAccidents
Shooting
Anomaly-Videos-Part-4
Vandalism
Stealing
Shoplifting


In [ ]:
df_prep = pd.DataFrame({'filename':[],'label':[]})
path = 'Originals/'
save_path = 'Preprocessed/'
path_len = len(path)
count = 0
for file in glob.glob(path + '*'):
    print(file)
    file_len = len(file)                                                                         # Calculating the length of the entire file path
    for file2 in glob.glob(path + file[path_len:] + '/' + '*.*'):                                # Extracting all the files with all type of extensions (only image files are supposed to be at this location)
#           print(file2)
        img = cv2.imread(file2)                                                                  # Reading the each image file using OpenCV
        count += 1
        if '.jpg' in file2:
          filename = 'Image_' + str(count) + '.jpg'                                            # Structuring the filenames
        elif '.jpeg' in file2:
          filename = 'Image_' + str(count) + '.jpeg'
        elif '.png' in file2:
          filename = 'Image_' + str(count) + '.png'
        elif '.gif' in file2:
          continue
        cv2.imwrite(save_path + filename,                                                    # Saving the image, specifying the new saving path
                    img)  
        df_prep = df_prep.append({'filename':filename,                                       # Appending the data to the preparation dataframe
                                  'label':file[10:]}, 
                                  ignore_index=True)



Originals\ants
Originals\bees


In [ ]:
df_prep

,filename,label
0,Image_1.jpg,ants
1,Image_2.jpg,ants
2,Image_3.jpg,ants
3,Image_4.jpg,ants
4,Image_5.jpg,ants
...,...,...
392,Image_394.jpg,bees
393,Image_395.jpg,bees
394,Image_396.jpg,bees
395,Image_397.jpg,bees


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

X = df_prep.drop('label', axis=1)
y = df_prep['label']
split = StratifiedShuffleSplit(n_splits = 10, test_size=0.3, random_state=42)
for train_index, test_index in split.split(X, y):
    X_train_str, X_test_str = X.iloc[train_index], X.iloc[test_index]
    y_train_str, y_test_str = y.iloc[train_index], y.iloc[test_index]

In [ ]:
train_data = pd.concat([X_train_str, y_train_str], axis = 1)

In [ ]:
train_data.label.value_counts()

bees    142
ants    135
Name: label, dtype: int64

In [ ]:
train_data

,filename,label
172,Image_173.jpg,ants
323,Image_325.jpg,bees
284,Image_286.jpg,bees
362,Image_364.jpg,bees
171,Image_172.jpg,ants
...,...,...
187,Image_189.jpg,ants
17,Image_18.jpg,ants
83,Image_84.jpg,ants
82,Image_83.jpg,ants


In [ ]:
test_data = pd.concat([X_test_str, y_test_str], axis = 1)

In [ ]:
test_data

,filename,label
196,Image_198.jpg,bees
282,Image_284.jpg,bees
261,Image_263.jpg,bees
50,Image_51.jpg,ants
245,Image_247.jpg,bees
...,...,...
43,Image_44.jpg,ants
18,Image_19.jpg,ants
19,Image_20.jpg,ants
5,Image_6.jpg,ants


In [ ]:
test_data.label.value_counts()

bees    62
ants    58
Name: label, dtype: int64

In [ ]:
df_training = train_data.sample(frac=1).reset_index(drop=True)
df_training['filename_new'] = 'Image_' + (df_training.index + 1).astype(str)+ '.jpg'

In [ ]:
df_testing = test_data.sample(frac=1).reset_index(drop=True)
df_testing['filename_new'] = 'Image_' + (df_testing.index + 1).astype(str)+ '.jpg'

In [ ]:
df_testing

,filename,label,filename_new
0,Image_262.jpg,bees,Image_1.jpg
1,Image_271.jpg,bees,Image_2.jpg
2,Image_125.jpg,ants,Image_3.jpg
3,Image_274.jpg,bees,Image_4.jpg
4,Image_206.jpg,bees,Image_5.jpg
...,...,...,...
115,Image_297.jpg,bees,Image_116.jpg
116,Image_284.jpg,bees,Image_117.jpg
117,Image_306.jpg,bees,Image_118.jpg
118,Image_198.jpg,bees,Image_119.jpg


In [ ]:
def copy_image(orig_path, final_path):
    img = cv2.imread(orig_path)
    cv2.imwrite(final_path, img)
    

testing_path = 'test/'
if not os.path.exists(testing_path):
    os.mkdir(testing_path)
    
training_path = 'train/'
if not os.path.exists(training_path):
    os.mkdir(training_path)
    
df_training.apply(lambda row: copy_image(save_path + row['filename'], training_path + row['filename_new']), axis=1)
df_testing.apply(lambda row: copy_image(save_path + row['filename'], testing_path + row['filename_new']), axis=1)

df_training.drop(columns=['filename']). \
    rename(columns={'filename_new':'filename','label':'label'})[['filename','label']]. \
    set_index('filename').to_csv('Training_set.csv')
df_testing.drop(columns=['filename']). \
    rename(columns={'filename_new':'filename','label':'label'})[['filename','label']]. \
    set_index('filename').to_csv('Testing_set_ans.csv')
df_testing.drop(columns=['filename']). \
    rename(columns={'filename_new':'filename','label':'label'})['filename']. \
    to_csv('Testing_set.csv', index=False)

print('Done')

Done
